<h1>Introduction</h1>
Clone all files, install all libraries

In [1]:
!pip install datasets torch torchaudio jiwer transformers openai-whisper faster-whisper soundfile librosa
# !sudo apt install python3-torch python3-torchaudio python3-jiwer python3-transformers python3-Openai-whisper 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 311.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 216.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 171.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 167.0 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 188.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 143.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 215.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 252.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 188.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 208.3 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 250.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89/89 [datasets]2m88/89 [datasets]orchaudio]sformers]ub]2]12]]


<h1>Get all global variables and libraries added</h1>


In [8]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import torch


device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)
print(torch_dtype)

cpu
torch.float32


In [9]:
audio_path = "../creolese-audio-dataset/finetune_eligible"
transcription_path = "../creolese-audio-dataset/finetune_eligible/transcripts.json"

# Add the pretrained Whisper model

In [10]:
whisper_model_id = "openai/whisper-base"
whisper_processor = WhisperProcessor.from_pretrained(whisper_model_id)

model = WhisperForConditionalGeneration.from_pretrained(whisper_model_id)

model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=False)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          

In [11]:
from transformers import pipeline

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=whisper_processor.tokenizer,
    feature_extractor=whisper_processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


Device set to use cpu


# Run pretrained evaluations

## Import libraries and load files

In [6]:
# pip install jiwer

In [12]:
import os
import json
import jiwer

with open(transcription_path, "r") as f:
    dataset = json.load(f)

## Officially run the evaluations
### Make and store the predictions

In [14]:
ground_truth = []
predictions= []

for item in dataset:
  filename= ""
  audio_path = audio_path
  filename = os.path.basename(item["audio"])
  audio_path_filename = os.path.join(audio_path, filename)
  reference = item["text"]

  try:
      result = asr_pipeline(audio_path_filename, return_timestamps = True)
      hypothesis = result["text"]
  except Exception as e:
      print(f"Skipping {audio_path}: {e}")
      continue

  ground_truth.append(reference.lower())
  predictions.append(hypothesis.lower())

output_data = [{"ref": ref, "hyp": hyp} for ref, hyp in zip(ground_truth, predictions)]

with open("../pretrained_predictions/whisper_pretrained_predictions.json", "w") as out_file:
    json.dump(output_data, out_file, indent=4)

/home/kris/Documents/fyp/fyp_env/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
/home/kris/Documents/fyp/fyp_env/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
/home/kris/Documents/fyp/fyp_env/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is d

### Normalize to enable JIWER processing
You can run this script from here once you have already transcribed your dataset (the predictions are stored on disk).

In [15]:
import json
import jiwer

with open("../pretrained_predictions/whisper_pretrained_predictions.json", "r") as f:
    data = json.load(f)

ground_truth = [entry['ref'] for entry in data]
predictions = [entry['hyp'] for entry in data]
        

In [16]:
def custom_transform(text):
        # Convert to lowercase
    text = text.lower()
    # Remove punctuation (basic approach)
    for char in ",.:;!?\"'()[]{}":
        text = text.replace(char, "")
    # Remove extra spaces
    text = " ".join(text.split())
    return text

# Apply transformation to each text individually
ground_truth_transformed = [custom_transform(text) for text in ground_truth]
predictions_transformed = [custom_transform(text) for text in predictions]

# === STEP 4: COMPUTE METRICS ===
wer = jiwer.wer(ground_truth_transformed, predictions_transformed)
cer = jiwer.cer(ground_truth_transformed, predictions_transformed)
mer = jiwer.mer(ground_truth_transformed, predictions_transformed)

# === STEP 5: PRINT ===
print(f"WER: {wer:.4f}")
print(f"CER: {cer:.4f}")
print(f"MER: {mer:.4f}")



WER: 1.3491
CER: 0.8563
MER: 0.9403
